<a href="https://colab.research.google.com/github/ballmdr/Forex-Prediction-Machine_Learning/blob/master/Forex_Pytorch_RNN_LSTM_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fxcmpy

     |████████████████████████████████| 204kB 8.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/bd/5d/9f/ebc0c223ec59641877c817eb10442627e69af88c126f1f53a8
Successfully built socketIO-client


In [0]:
import torch
import torch.nn as nn
import fxcmpy
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

def connect():
    return fxcmpy.fxcmpy(access_token=access_token, server=server)

In [0]:
access_token = '82e41ea2bcf020754f67aeb835e9ad617dbc5427'
server = 'demo'
symbol = 'BTCUSD'
symbol2 = 'BTC/USD'
target_windows = 5
digits = 1
ratio = 100
n_prices = 2000

In [4]:
con = connect()

|WARNING|2019-07-28 01:59:48,481|Default account set to 96112465, to change use set_default_account().


In [0]:
def dfProcess(df):
# Short-Term
# ใช้ค่า 3, 5, 8, 10, 12, or 15.
# Long-Term
# ใช้ค่า 30, 35, 40, 45, 50, or 60.
  df['ma3'] = np.round(df.close.rolling(3).mean(), 5)
  df['ma5'] = np.round(df.close.rolling(5).mean(), 5)
  df['ma8'] = np.round(df.close.rolling(8).mean(), 5)
  df['ma10'] = np.round(df.close.rolling(10).mean(), 5)
  df['ma12'] = np.round(df.close.rolling(12).mean(), 5)
  df['ma15'] = np.round(df.close.rolling(15).mean(), 5)
  #df['ma20'] = np.round(df.close.rolling(20).mean(), 5)
  #df['ma7_shift3'] = df.ma7.shift(-3)
  #df['ma10_shift3'] = df.ma10.shift(-3)
  #df['close_shift3'] = df.close.shift(-3)
  df['close_diff_pip'] = - df.close.diff(-target_windows) * digits
  
  arr = ['0', '1', '2', '3', '4']
  for i in range(1, target_windows):
    df[arr[i]] = - df.close.diff(-i) * digits
    
  df.dropna(inplace=True)

  #df['close_target_percent'] = np.round((df['close_diff_pip']*digits)/df.close, 2) * 100
  #df['7_10_target_percent'] = np.round(((df['ma7_shift3']-df['ma10_shift3'])*digits)/df['ma10_shift3'], 2)
  #df['5_7_target_percent'] = np.round(((df.ma5-df.ma7)*digits)/df.ma7, 2)
  

  df['Target'] = np.ones(len(df))
  # df['Target'].loc[(df['close_target_percent'] > 9) & (df['7_10_target_percent'] > 1) & (df['5_7_target_percent'] > 1)] = 2
  # df['Target'].loc[(df['close_target_percent'] < -9) & (df['7_10_target_percent'] < -1) & (df['5_7_target_percent'] < -1)] = 0
  df['Target'].loc[(df['close_diff_pip'] > ratio) &
                   (df['1'] > -ratio) &
                   (df['2'] > -ratio) &
                   (df['3'] > -ratio) &
                   (df['4'] > -ratio)
                  ] = 2
  df['Target'].loc[(df['close_diff_pip'] < -ratio) &
                   (df['1'] < ratio) &
                   (df['2'] < ratio) &
                   (df['3'] < ratio) &
                   (df['4'] < ratio)                    
                  ] = 0
  print(df.Target.value_counts())

  df.drop(['bidopen', 'bidclose', 'bidhigh', 'askopen', 'askclose', 'askhigh', 'bidlow', 'asklow', 'tickqty'], axis=1, inplace=True)
  df.dropna(inplace=True)
  print(df.head())
  print(df.shape)
  
  print(df.info())
  
  return df


def getXY(df, windows):

  X = []
  Y = []
  for i in range(len(df)-(windows+1)):
    

    fig = plt.figure(frameon=False) 
    fig.set_size_inches(3,2)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)

    _df = df.iloc[i:windows]
    plt.plot(_df.ma3, color='red')
    plt.plot(_df.ma5, color='blue')
    plt.plot(_df.ma8, color='green')
    plt.plot(_df.ma10, color='orange')
    plt.plot(_df.ma12, color='black')
    plt.plot(_df.ma15, color='purple')
    #plt.plot(_df.open, color='black')
   # plt.plot(_df.close, color='purple')

    plt.savefig('tmp2.png')

    plt.close()

    img = plt.imread('tmp2.png')
    new_img = resize(img, (100,150))
    #print(new_img.shape)
    if i % 1000 == 0:
      print(i)
      plt.imshow(new_img)
      plt.show()
      print(i, ' - ', windows)
      print('target: ', df.Target.iloc[windows-1])
    X.append(new_img)
    Y.append(df.Target.iloc[windows-1])

    windows += 1

  return X, Y


In [58]:
df = con.get_candles(symbol2, period='H4', number=n_prices)
df['close'] = (df.bidclose + df.askclose) / 2
df = dfProcess(df)

#df['close'] = (df.close - df.close.mean()) / (df.close.max() - df.close.min())
#print(df['close'].head())

split = int(len(close) * 0.60)
train_data = df.iloc[:split].drop(['Target'], axis=1).values
train_target = df.iloc[:split]['Target'].values
test_data = df.iloc[split:].drop(['Target'], axis=1).values
test_target = df.iloc[split:]['Target'].values

minmax = MinMaxScaler()
train_data = minmax.fit_transform(train_data)
test_data = minmax.transform(test_data)
print(train_data.shape)
print(train_target.shape)
print(test_data.shape)
print(test_target.shape)

2.0    191
1.0    181
0.0    115
Name: Target, dtype: int64
                       close         ma3      ma5  ...       3       4  Target
date                                               ...                        
2019-04-02 02:00:00  4169.00  4147.41667  4138.55  ...  651.00  583.75     2.0
2019-04-02 06:00:00  4760.00  4354.83333  4267.00  ...   -7.25  152.75     2.0
2019-04-02 10:00:00  4810.00  4579.66667  4402.45  ...  102.75  275.75     2.0
2019-04-02 14:00:00  4820.00  4796.66667  4538.90  ...  265.75  166.25     2.0
2019-04-02 18:00:00  4752.75  4794.25000  4662.35  ...  233.50  230.50     2.0

[5 rows x 13 columns]
(487, 13)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 487 entries, 2019-04-02 02:00:00 to 2019-07-26 02:00:00
Data columns (total 13 columns):
close             487 non-null float64
ma3               487 non-null float64
ma5               487 non-null float64
ma8               487 non-null float64
ma10              487 non-null float64
ma12             

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [59]:
test_target

array([1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 1., 1., 2., 2., 1., 2., 2.,
       2., 1., 2., 2., 2., 2., 2., 2., 2., 2., 1., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 1., 0., 0., 0., 0., 0., 1., 1., 2., 1., 1., 1., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 1., 2., 2., 2., 2., 2., 1., 2., 2., 2., 0.,
       0., 1., 0., 0., 1., 2., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 1.,
       1., 1., 2., 2., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
       2., 1., 1., 1., 1., 0., 2., 2., 1., 0., 0., 0., 0., 0., 0., 1., 2.,
       2., 2., 2., 1., 1., 2., 2., 2., 2., 1., 0., 0., 1., 1., 1., 2., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 2., 2., 2., 1.,
       2., 0., 0., 0., 0., 0., 1., 1.])

In [75]:
class RNN(nn.Module):
    def __init__(self, hidden, K, input_size):
        super(RNN, self).__init__()
        
        self.linearH = nn.Linear(input_size + hidden, hidden)
        self.linearO = nn.Linear(input_size + hidden, K)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, h):
        x = torch.cat((x, h), 1)
        h = torch.tanh(self.linearH(x))
        out = self.softmax(self.linearO(x))
        
        return out, h
      
      
class LSTM(nn.Module):
  def __init__(self, hidden, output_size, vocab_size):
    super(LSTM, self).__init__()
    
    self.linearFt = nn.Linear(vocab_size + hidden, hidden)
    self.linearIt = nn.Linear(vocab_size + hidden, hidden)
    self.linearCt1 = nn.Linear(vocab_size + hidden, hidden)
    self.linearOt = nn.Linear(vocab_size + hidden, hidden)
    self.linearO = nn.Linear(vocab_size + hidden, output_size)
    
    self.softmax = nn.LogSoftmax(dim=1)
    
  def forward(self, x, h, c):
    combined = torch.cat((x, h), 1)
    ft = torch.sigmoid(self.linearFt(combined))
    it = torch.sigmoid(self.linearIt(combined))
    ct1 = torch.tanh(self.linearCt1(combined))
    c = (ft*c) + (it * ct1)
    
    ot = torch.sigmoid(self.linearOt(combined))
    h = ot * torch.tanh(c)
    
    out = self.softmax(self.linearO(combined))
    
    return out, h, c

  
class GRU(nn.Module):
  
  def __init__(self, hidden, output_size, input_size):
    super(GRU, self).__init__()
    
    self.linearZt = nn.Linear(input_size + hidden, hidden)
    self.linearRt = nn.Linear(input_size + hidden, hidden)
    self.linearHt = nn.Linear(input_size + hidden, hidden)
    
    self.linearO = nn.Linear(input_size + hidden, output_size)
    
    self.softmax = nn.LogSoftmax(dim=1)
    
  def forward(self, x, h):
    combined = torch.cat((x, h), 1)
    zt = torch.sigmoid(self.linearZt(combined))
    rt = torch.sigmoid(self.linearRt(combined))
    combined2 = torch.cat((x, rt*h), 1)
    ht = torch.tanh(self.linearHt(combined2))
    h = (1-zt) * h + zt * ht
    
    out = self.softmax(self.linearO(combined))
  
    return out, h
  
windows = 6
input_size = 12
hidden = 128
K = 3
learning_rate = 0.05
epochs = 15000
train_loss = []
test_loss = []


model = GRU(hidden, K, input_size)
model.cuda()
loss_fn = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[2000, 4000, 6000, 8000, 10000, 12000, 14000], gamma=0.1)
N = len(train_data)-(windows+1)
print(N)

for e in range(epochs):
    model.train()
    running_loss = 0.0
    testing_loss = 0.0
    
    
    for i in range(N):
        
        inputs = train_data[i:i+windows]
        y = train_target[(i+windows)-1]
        #print(i, ' Input: ', inputs, ' Target: ', y, ' ', train_data[windows-1])
        
        h = torch.zeros([1, hidden], dtype=torch.float32, device='cuda')
        c = torch.zeros([1, hidden], dtype=torch.float32, device='cuda')
        
        y_tensor = torch.tensor(y, dtype=torch.long, device='cuda')
        
        #print('y: ', y_tensor)
        
        optimizer.zero_grad()
        
        for w in inputs:
          #print(w)
          x_tensor = torch.tensor(w, dtype=torch.float32, device='cuda')
          x_tensor = x_tensor.view(1,-1)
          out, h = model(x_tensor, h)

        #print('out: ', out)
        loss = loss_fn(out, y_tensor.unsqueeze(0))
        loss.backward(retain_graph=True)
        optimizer.step()
        running_loss += loss.item()
        

    run_loss = running_loss / N
    train_loss.append(run_loss)
    scheduler.step()

    model.eval()
    test_count = 5
    for j in range(test_count):
        index = np.random.randint(0, len(test_data)-(windows+1))
        #print('index: ', index)
        h = torch.zeros([1, hidden], dtype=torch.float32, device='cuda')
        c = torch.zeros([1, hidden], dtype=torch.float32, device='cuda')
        
        inputs = test_data[index:index+windows]
        y = test_target[(index+windows)-1]
        y_tensor = torch.tensor(y, dtype=torch.long, device='cuda')

        for w in inputs:
            x = torch.tensor(w.reshape(1,-1), dtype=torch.float32, device='cuda')
            out, h = model(x, h)

        loss = loss_fn(out, y_tensor.unsqueeze(0))
        testing_loss += loss.item()
    
    avg_loss = testing_loss / test_count
    test_loss.append(avg_loss)
    
    if e % 100 == 0:
        print(e, ' Train Loss: ', run_loss, ' Test Loss: ', avg_loss)      

285
0  Train Loss:  1.608080075916491  Test Loss:  1.9495830059051513


KeyboardInterrupt: ignored

In [62]:
y_tensor

tensor([[2]], device='cuda:0')

In [66]:
y_tensor.cpu().detach().numpy()[0][0]

2